In [1]:
import requests
import json
import time
from PIL import Image
import pandas as pd

In [2]:
url = 'http://ipinfo.io/json'

In [3]:
loc_json = requests.get(url)

In [4]:
my_loc = json.loads(loc_json.content)

In [5]:
curr_location = my_loc['loc']

In [6]:
curr_location

'38.7797,-77.1866'

In [7]:
class GooglePlaces(object):
    def __init__(self, apikey):
        super(GooglePlaces, self).__init__()
        self.apikey = apikey
    
    def search_places_by_coordinate(self, location, radius, types):
        endpoint_rul = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
        places = []
        params = {
            'location' : location,
            'radius' : radius,
            'type' : types,
            'key' : self.apikey}
        
        res = requests.get(endpoint_rul, params = params)
        print(res)
        
        # loading to the python dictionary using json.loads function
        
        results = json.loads(res.content)
        # return results
        
        places.extend(results['results'])
        time.sleep(2)
        while 'next_page_token' in results:
            params['pagetoken'] = results['next_page_token'],
            res = requests.get(endpoint_rul, params=params)
            results = json.loads(res.content)
            places.extend(results['results'])
            time.sleep(2)
        return places
    
    def get_place_details(self, place_id, fields):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/details/json?"
        params = {
            'placeid': place_id,
            'fields': ",".join(fields),
            'key': self.apikey
        }
        res = requests.get(endpoint_url, params = params)
        place_details =  json.loads(res.content)
        return place_details
    

In [8]:
api = GooglePlaces('*****')

In [9]:
places = api.search_places_by_coordinate(curr_location,  "3000", "restaurant")

<Response [200]>


In [10]:
len(places)

60

In [11]:
fields = ['name', 'user_ratings_total', 'formatted_address','geometry', 'international_phone_number', 'website', 'rating', 'review', 'photo']

In [12]:
places[0]['name']

'Silver Diner'

In [13]:
import googlemaps

In [14]:
api_key = '*****'

In [15]:
gmaps = googlemaps.Client(key = api_key)

In [26]:
places_df = pd.DataFrame({'name':[],'address':[],'geometry':[],'phone_number':[],
                       'website':[], 'total_user_ratings':[], 'reviews':[]})

In [27]:
for place in places:
    details = api.get_place_details(place['place_id'], fields)
    print(details['result']['geometry'])
    

{'location': {'lat': 38.7775425, 'lng': -77.17359239999999}, 'viewport': {'northeast': {'lat': 38.7789029802915, 'lng': -77.1723838697085}, 'southwest': {'lat': 38.7762050197085, 'lng': -77.1750818302915}}}
{'location': {'lat': 38.7728444, 'lng': -77.17861940000002}, 'viewport': {'northeast': {'lat': 38.77421098029151, 'lng': -77.1772830197085}, 'southwest': {'lat': 38.77151301970851, 'lng': -77.17998098029152}}}
{'location': {'lat': 38.78111879999999, 'lng': -77.185665}, 'viewport': {'northeast': {'lat': 38.78252468029149, 'lng': -77.1845266197085}, 'southwest': {'lat': 38.7798267197085, 'lng': -77.18722458029151}}}
{'location': {'lat': 38.773536, 'lng': -77.1830657}, 'viewport': {'northeast': {'lat': 38.7749116802915, 'lng': -77.1818214197085}, 'southwest': {'lat': 38.7722137197085, 'lng': -77.1845193802915}}}
{'location': {'lat': 38.7750714, 'lng': -77.17107779999999}, 'viewport': {'northeast': {'lat': 38.7763574802915, 'lng': -77.16982036970849}, 'southwest': {'lat': 38.77365951970

In [28]:
for place in places:
    details = api.get_place_details(place['place_id'], fields)
    
    try:
        website = details['result']['website']
    except KeyError:
        website = ""
        
    try:
        name = details['result']['name']
    except KeyError:
        name = ""
    try:
        geometry = details['result']['geometry']
    except:
        geometry = ""
    
    try:
        user_ratings_total = details['result']['user_ratings_total']
    except KeyError:
        user_ratings_total = ""
        
    try:
        address = details['result']['formatted_address']
    except KeyError:
        address = ""
        
    try:
        phone_number = details['result']['international_phone_number']
    except KeyError:
        phone_number = ""
        
    try:
        reviews = details['result']['reviews']
    except KeyError:
        reviews = []
        
    places_df.loc[len(places_df)] = [name, address, geometry, phone_number, website, user_ratings_total, reviews]
        
    print('=======================PLACE========================')
    print('Name:', name)
    print('Total User Ratings:', user_ratings_total)
    print('Website:', website)
    print('Address:', address)
    print('Phone Number:', phone_number)
    print('=======================REVIEWS======================')
    for review in reviews:
        author_name = review['author_name']
        rating = review['rating']
        text = review['text']
        time = review['relative_time_description']
        profile_photo = review['profile_photo_url']
        print("Author Name:", author_name)
        print("Rating:", rating)
        print("Text:", text)
        print("Time:", time)
        print("Profile photo:", profile_photo)
        print("------------------------------------------------")

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:883: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


=======================PLACE========================
Name: Silver Diner
Total User Ratings: 2482
Website: http://www.silverdiner.com/
Address: 6592 Springfield Mall, Springfield, VA 22150, USA
Phone Number: +1 703-924-1701
=======================REVIEWS======================
Author Name: Hilda Kroll
Rating: 5
Text: Great place for comfort food, especially on a cold, rainy day.  My friend got the combo salad and chili.  I got the Ribeye Philly Steak on a sweet potato sub roll.  The side of honey pepper relish was something different and really good.  Looks like a real, old fashioned diner like the one I knew as a teenager.  Now, I appreciate the ten percent discount for seniors.
Time: 5 months ago
Profile photo: https://lh3.googleusercontent.com/a/ALm5wu3KGRjm1HlFubHchXb-qe4Am5wpKuzTMu0W_djM=s128-c0x00000000-cc-rp-mo-ba7
------------------------------------------------
Author Name: Cecilia Sterling
Rating: 4
Text: I had the lemony hummus with cauliflower pita bread and it was delicious.

In [29]:
places_df.iloc[1]['geometry']

{'location': {'lat': 38.7728444, 'lng': -77.17861940000002},
 'viewport': {'northeast': {'lat': 38.77421098029151,
   'lng': -77.1772830197085},
  'southwest': {'lat': 38.77151301970851, 'lng': -77.17998098029152}}}

In [30]:
places_df.iloc[10]['website']

'https://locations.outback.com/virginia/springfield/6651-backlick-road'

In [31]:
places_df

,name,address,geometry,phone_number,website,total_user_ratings,reviews
0,Silver Diner,"6592 Springfield Mall, Springfield, VA 22150, USA","{'location': {'lat': 38.7775425, 'lng': -77.17...",+1 703-924-1701,http://www.silverdiner.com/,2482,"[{'author_name': 'Hilda Kroll', 'author_url': ..."
1,Thai Cafe,"6701 Loisdale Rd E, Springfield, VA 22150, USA","{'location': {'lat': 38.7728444, 'lng': -77.17...",+1 703-922-4942,http://www.thaicafespringfield.com/,333,"[{'author_name': 'Anne M.', 'author_url': 'htt..."
2,PaperMoon Springfield,"6315 Amherst Ave, Springfield, VA 22150, USA","{'location': {'lat': 38.78111879999999, 'lng':...",+1 703-866-4160,http://papermoonvip.com/,229,"[{'author_name': 'Charles Lundgate', 'author_u..."
3,Chick-fil-A,"6681A Backlick Rd, Springfield, VA 22150, USA","{'location': {'lat': 38.773536, 'lng': -77.183...",+1 703-644-0155,https://www.chick-fil-a.com/springfieldinline,2344,"[{'author_name': 'Winter Bray', 'author_url': ..."
4,Bertucci's Italian Restaurant,"6525 Frontier Dr, Springfield, VA 22150, USA","{'location': {'lat': 38.7750714, 'lng': -77.17...",+1 703-313-6700,https://locations.bertuccis.com/us/va/springfi...,748,"[{'author_name': 'Petter Llanos', 'author_url'..."
5,Hard Times Cafe,"6362 Springfield Blvd, Springfield, VA 22150, USA","{'location': {'lat': 38.7819258, 'lng': -77.18...",+1 703-913-5600,https://www.hardtimes.com/locations/springfield/,727,"[{'author_name': 'MkMG09', 'author_url': 'http..."
6,Five Guys,"6541 Backlick Rd, Springfield, VA 22150, USA","{'location': {'lat': 38.7774695, 'lng': -77.18...",+1 571-297-9344,https://restaurants.fiveguys.com/6541-backlick...,818,"[{'author_name': 'Patricia Torcello Catalina',..."
7,Chipotle Mexican Grill,"7108 Old Keene Mill Rd, Springfield, VA 22150,...","{'location': {'lat': 38.7791265, 'lng': -77.18...",+1 703-644-6982,https://locations.chipotle.com/va/springfield/...,378,"[{'author_name': 'Jefferson Carazo', 'author_u..."
8,Delia's Pizzeria & Grille,"6715-F, Backlick Rd, Springfield, VA 22150, USA","{'location': {'lat': 38.7717596, 'lng': -77.18...",+1 703-451-0242,http://deliasspringfield.com/,783,"[{'author_name': 'Dani Elle', 'author_url': 'h..."
9,Subway,"6711 Spring Mall Dr Unit # 3, Springfield, VA ...","{'location': {'lat': 38.7711371, 'lng': -77.17...",+1 703-719-6562,https://restaurants.subway.com/united-states/v...,149,"[{'author_name': 'Ay Yo Cuzz Ay Yo Sann', 'aut..."


In [22]:
places_df.iloc[3]['reviews'][0]

{'author_name': 'Charles Lundgate',
 'author_url': 'https://www.google.com/maps/contrib/104040028487482193917/reviews',
 'language': 'en',
 'original_language': 'en',
 'profile_photo_url': 'https://lh3.googleusercontent.com/a/ALm5wu0P8pxEtCWcKi-oyo88_lcAp3YSGvYdNjrvdoIq=s128-c0x00000000-cc-rp-mo',
 'rating': 3,
 'relative_time_description': '7 months ago',
 'text': 'Decent quality dancers and nice venue. Very pricey, however. 20 dollar cover, 10 dollars for a non alcoholic drink. 12 percent charge to use atm and 24 percent fee on credit card use.\n\nDance prices are also very high. 100 for a 2 song lap dance that didn’t include any contact. Not too sure if that was an actual club rule or me getting ripped off by the dancer. Be very judicious about who you select as a dancer. Some have poor attitudes and expect a free check for no work at all. Originally, the dancer mentioned she doesn’t even do 100 dollar dances like she was above it. At that point, I should’ve caught the red flags and

In [32]:
places_df.to_csv('places_df.csv', index=False)  